In [3]:
from datetime import datetime
from pathlib import Path
from typing import Any, Dict, Union

from hyperopt import STATUS_OK, Trials, fmin, tpe
from loguru import logger
from mads_datasets import DatasetFactoryProvider, DatasetType
import mlflow
from mltrainer import ReportTypes, Trainer, TrainerSettings, metrics
from mltrainer.preprocessors import PaddedPreprocessor
from pydantic import BaseModel
import torch
import torch.nn as nn
import torch.optim as optim

In [4]:
preprocessor = PaddedPreprocessor()
gestures_dataset_factory = DatasetFactoryProvider.create_factory(
    DatasetType.GESTURES
)
streamers = gestures_dataset_factory.create_datastreamer(
    batchsize=64,
    preprocessor=preprocessor,
)
train = streamers["train"]
valid = streamers["valid"]
train_streamer = train.stream()
valid_streamer = valid.stream()

2026-01-08 13:07:22.592 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /Users/koendirkvanesterik/.cache/mads_datasets/gestures
100%|██████████| 651/651 [00:00<00:00, 4022.50it/s]


In [5]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("mlflow-gestures-rnn-hyperopt")

2026/01/08 13:07:24 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/01/08 13:07:24 INFO mlflow.store.db.utils: Updating database tables
2026-01-08 13:07:24 INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
2026-01-08 13:07:24 INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2026-01-08 13:07:24 INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
2026-01-08 13:07:24 INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/Users/koendirkvanesterik/Projects/mads/mads-ml-koenvanesterik/notebooks/rnns/mlruns/1', creation_time=1767865600314, experiment_id='1', last_update_time=1767865600314, lifecycle_stage='active', name='mlflow-gestures-rnn-hyperopt', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [6]:
class ModelConfig(BaseModel):
    input_size: int
    hidden_size: int
    num_layers: int
    output_size: int
    dropout: float = 0.0


class GRU(nn.Module):
    def __init__(
        self,
        config: ModelConfig,
    ) -> None:
        super().__init__()
        self.config = config
        self.rnn = nn.GRU(
            input_size=config.input_size,
            hidden_size=config.hidden_size,
            dropout=config.dropout,
            batch_first=True,
            num_layers=config.num_layers,
        )
        self.linear = nn.Linear(config.hidden_size, config.output_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x, _ = self.rnn(x)
        last_step = x[:, -1, :]
        yhat = self.linear(last_step)
        return yhat

In [17]:
settings = TrainerSettings(
    epochs=20,
    metrics=[metrics.Accuracy()],
    logdir="modellogs",
    train_steps=100,
    valid_steps=100,
    reporttypes=[ReportTypes.MLFLOW, ReportTypes.TOML],
)

In [8]:
def objective(params: Dict[str, Union[int, float]]) -> Dict[str, Any]:
    with mlflow.start_run():
        mlflow.set_tag("model", "recurrent-neural-network")
        mlflow.set_tag("dev", "vanesterik")
        mlflow.log_params(params)

        model = GRU(ModelConfig(**params))
        trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=nn.CrossEntropyLoss(),
            optimizer=optim.Adam,
            traindataloader=train_streamer,
            validdataloader=valid_streamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau,
            device=torch.device("mps"),
        )
        trainer.loop()

        tag = datetime.now().strftime("%Y%m%d-%H%M")
        models_dir = Path("models").resolve()

        if not models_dir.exists():
            models_dir.mkdir()
            logger.info(f"Created {models_dir}")

        models_path = models_dir / (tag + "model.pt")
        torch.save(model, models_path)

        mlflow.log_artifact(
            local_path=models_path, artifact_path="pytorch_models"
        )

        return {"loss": trainer.test_loss, "status": STATUS_OK}


In [9]:
search_space = {
    "input_size": 3,
    "hidden_size": 64,
    "num_layers": 1,
    "output_size": 20,
    "dropout": 0.1,
}

In [18]:
results = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=1,
    trials=Trials(),
)

  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

2026-01-08 13:15:09.138 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs/20260108-131509
2026-01-08 13:15:09.142 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|##########| 100/100 [00:01<00:00, 53.71it/s]
2026-01-08 13:15:11.640 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.7283 test 2.3462 metric ['0.1308']
100%|##########| 100/100 [00:01<00:00, 55.90it/s]
2026-01-08 13:15:14.053 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 2.2117 test 2.1272 metric ['0.2278']
100%|##########| 100/100 [00:01<00:00, 55.51it/s]
2026-01-08 13:15:16.484 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 1.9383 test 1.7916 metric ['0.3420']
100%|##########| 100/100 [00:01<00:00, 54.89it/s]
2026-01-08 13:15:18.937 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 1.6341 test 1.4856 metric ['0.5170']
100%|##########| 100/100 [00:01<00:00, 55.13it/s]
2026-01-08 

100%|██████████| 1/1 [00:49<00:00, 49.50s/trial, best loss: 0.09726429749280215]


In [11]:
logger.info(f"\n\n{results}")

2026-01-08 13:09:57.449 | INFO     | __main__:<module>:1 - 

{}
